In [13]:
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin

In [14]:
df = pd.read_csv('../../../../raw_data/data_prep.csv')

In [15]:
train, dev = train_test_split(df, test_size=0.1, random_state=42)

In [16]:
train = train.reset_index(drop=True)
dev = dev.reset_index(drop=True)

In [17]:
train.shape, dev.shape

((1080, 8), (120, 8))

In [18]:
categories = ['A', 'AAA', 'AA+', 'A-', 'A+', 'AA-', 'AA']

In [19]:
dev = dev.query('rating in @categories')
train = train.query('rating in @categories')

In [20]:
train = train.reset_index(drop=True)
dev = dev.reset_index(drop=True)

In [21]:
nlp = spacy.load('ru_core_news_lg')

In [22]:
def create_docbin(data):
    db = DocBin()
    global categories
    for i in range(data.shape[0]):    
        doc = nlp.make_doc(str(data["tokenized_str"][i]))
        doc.cats = {category: 0 for category in categories}
        doc.cats[data["rating"][i]] = 1
        db.add(doc)
    return db

In [23]:
create_docbin(dev).to_disk("dev.spacy")
create_docbin(train).to_disk("train.spacy")

In [24]:
! python -m spacy train config.cfg --output ./subsym_model --paths.train train.spacy --paths.dev dev.spacy

ℹ Saving to output directory: subsym_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       54.97    0.55
  0     200          24.80       76.29    0.76
  0     400          21.87       76.31    0.76
  1     600          19.66       81.54    0.82
  1     800           9.76       83.43    0.83
  1    1000          10.45       85.58    0.86
  2    1200          11.30       83.81    0.84
  2    1400           7.06       87.28    0.87
  2    1600           6.03       86.97    0.87
  3    1800           5.47       87.59    0.88
  3    2000           3.28       86.14    0.86
  3    2200           5.79       87.53    0.88
  4    2400           2.72       86.48    0